In [26]:
import os
import h3
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import timedelta
from tqdm import tqdm

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

from mirrorverse.utils import read_data_w_cache

In [ ]:
data = read_data_w_cache(
    'select * from movement_model_simulation_m9_a2_v9_r3'
)
data["lat"] = data["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[0])
data["lon"] = data["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[1])
print(data.shape)
data.head()

In [18]:
df = data.sort_values("time")
df['log_quanta'] = np.log(df['_quanta'] + 0.001) / np.log(10)


# Create the animated heatmap
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    color="log_quanta",
    animation_frame="time",
    #radius=10,  # Adjust for smoothing
    center={"lat": 51, "lon": 176},
    zoom=2,
    mapbox_style="carto-positron",  # Alternative: "carto-positron"
    title="Heatmap Animation of log(_quanta) Over Time",
    height=600,
    range_color=[-1, 2],#df['log_quanta'].max()],
    color_continuous_scale="plasma"
)



# Show the figure
fig.show()

In [20]:
df['log_quanta'].max()

In [21]:
10 ** 2.8

# Version 2

In [ ]:
diffusion = read_data_w_cache(
    'select * from movement_model_diffusion_m9_a2_v9_t3'
)
diffusion = diffusion[[c for c in diffusion.columns if c != 'time']]
print(diffusion.shape)
diffusion.head()

In [ ]:
df = diffusion[diffusion['date'] == diffusion['date'].min()][['h3_index', 'date']].drop_duplicates()

df["lat"] = df["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[0])
df["lon"] = df["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[1])
df = df[(df['lon'] > -178) & (df['lon'] < -157) & (df['lat'] > 55)]
del df['lat']
del df['lon']

df['_quanta'] = 10
df

dfs = []
dfs.append(df)

for _ in tqdm(list(range(31))):
    df_next = (
        df.rename({'h3_index': 'origin_h3_index'}, axis=1).merge(diffusion, on=['origin_h3_index', 'date'])
    )
    df_next['_quanta'] = df_next['_quanta'] * df_next['probability']
    df_next = df_next.groupby(['h3_index', 'date'])[['_quanta']].sum().reset_index()
    df_next['date'] = df_next['date'] + timedelta(days=1)
    df = df_next
    dfs.append(df)

df = pd.concat(dfs)
print(df.shape)
df.head()

In [ ]:
df["lat"] = df["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[0])
df["lon"] = df["h3_index"].apply(lambda x: h3.h3_to_geo(str(x))[1])
df = df.sort_values("date")
df['log_quanta'] = np.log(df['_quanta'] + 0.001) / np.log(10)


# Create the animated heatmap
fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    color="log_quanta",
    animation_frame="date",
    #radius=10,  # Adjust for smoothing
    center={"lat": 51, "lon": 176},
    zoom=2,
    mapbox_style="carto-positron",  # Alternative: "carto-positron"
    title="Heatmap Animation of log(_quanta) Over Time",
    height=600,
    range_color=[-1, 2],#df['log_quanta'].max()],
    color_continuous_scale="plasma"
)



# Show the figure
fig.show()